In [4]:
# imports
from scipy import io, signal
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [5]:
import pandas as pd
from numpy.linalg import LinAlgError
from statsmodels.tsa.stattools import adfuller
#1
def AE(x): # Absolute Energy
    x = np.asarray(x)
    return sum(x * x)

#2
def SM2(y):
    #t1 = time.time()
    f, Pxx_den = signal.welch(y)
    sm2 = 0
    n = len(f)
    for i in range(0,n):
        sm2 += Pxx_den[i]*(f[i]**2)
    
    #t2 = time.time()
    #print('time: ', t2-t2)
    return sm2


#3
def LOG(y):
    n = len(y)
    return np.exp(np.sum(np.log(np.abs(y)))/n)

#4
def WL(x): # WL in primary manuscript
    return np.sum(abs(np.diff(x)))

#5
def ADF(x): # teststat, pvalue, usedlag
    # augmented dickey fuller
    # returns a tuple
    res = None
    try:
        res = adfuller(x)
    except LinAlgError:
        res = np.NaN, np.NaN, np.NaN
    except ValueError:  # occurs if sample size is too small
        #print('Length Error')
        res = np.NaN, np.NaN, np.NaN

    return res[0], res[1], res[2]


#6
def AC(x, lag=5): # autocorrelation

    """
     [1] https://en.wikipedia.org/wiki/Autocorrelation#Estimation

    """
    # This is important: If a series is passed, the product below is calculated
    # based on the index, which corresponds to squaring the series.
    if type(x) is pd.Series:
        x = x.values
    if len(x) < lag:
        return np.nan
    # Slice the relevant subseries based on the lag
    y1 = x[:(len(x)-lag)]
    y2 = x[lag:]
    # Subtract the mean of the whole series x
    x_mean = np.mean(x)
    # The result is sometimes referred to as "covariation"
    sum_product = np.sum((y1-x_mean)*(y2-x_mean))
    # Return the normalized unbiased covariance
    return sum_product / ((len(x) - lag) * np.var(x))

#7
def BE(x, max_bins=30): # binned entropy
    hist, bin_edges = np.histogram(x, bins=max_bins)
    probs = hist / len(x)
    return - np.sum(p * np.math.log(p) for p in probs if p != 0)

#8
def C3(x, lag = 5): # c3 feature
    n = len(x)
    x = np.asarray(x)
    if 2 * lag >= n:
        return 0
    else:
        return np.mean((np.roll(x, 2 * -lag) * np.roll(x, -lag) * x)[0:(n - 2 * lag)])
    
    





#12
def AAC(x): #AAC in primary manuscript
    return np.mean(abs(np.diff(x)))

#13
def MSDC(x): # mean second derivative central
    diff = (np.roll(x, 1) - 2 * np.array(x) + np.roll(x, -1)) / 2.0
    return np.mean(diff[1:-1])

#14
def ZC(x, m = 0): # zero/mean crossing
    # m = np.mean(x)
    x = np.asarray(x)
    x = x[x != m]
    return sum(np.abs(np.diff(np.sign(x - m))))/2


#15
def SE(x): # sample entropy
    """
    [1] http://en.wikipedia.org/wiki/Sample_Entropy
    [2] https://www.ncbi.nlm.nih.gov/pubmed/10843903?dopt=Abstract
    """
    x = np.array(x)

    sample_length = 1 # number of sequential points of the time series
    tolerance = 0.2 * np.std(x) # 0.2 is a common value for r - why?

    n = len(x)
    prev = np.zeros(n)
    curr = np.zeros(n)
    A = np.zeros((1, 1))  # number of matches for m = [1,...,template_length - 1]
    B = np.zeros((1, 1))  # number of matches for m = [1,...,template_length]

    for i in range(n - 1):
        nj = n - i - 1
        ts1 = x[i]
        for jj in range(nj):
            j = jj + i + 1
            if abs(x[j] - ts1) < tolerance:  # distance between two vectors
                curr[jj] = prev[jj] + 1
                temp_ts_length = min(sample_length, curr[jj])
                for m in range(int(temp_ts_length)):
                    A[m] += 1
                    if j < n - 1:
                        B[m] += 1
            else:
                curr[jj] = 0
        for j in range(nj):
            prev[j] = curr[j]

    N = n * (n - 1) / 2
    B = np.vstack(([N], B[0]))

    # sample entropy = -1 * (log (A/B))
    similarity_ratio = A / B
    se = -1 * np.log(similarity_ratio)
    se = np.reshape(se, -1)
    return se[0]

#16
def TRAS(x, lag=5):
    # time reversal asymmetry statistic
    """
    |  [1] Fulcher, B.D., Jones, N.S. (2014).
    |  Highly comparative feature-based time-series classification.
    |  Knowledge and Data Engineering, IEEE Transactions on 26, 3026–3037.
    """
    n = len(x)
    x = np.asarray(x)
    if 2 * lag >= n:
        return 0
    else:
        return np.mean((np.roll(x, 2 * -lag) * np.roll(x, 2 * -lag) * np.roll(x, -lag) -
                        np.roll(x, -lag) * x * x)[0:(n - 2 * lag)])
    
    
#17    
def VAR(x): # variance 
    return np.var(x)

In [6]:
import time
fx = []
y_b = []

In [14]:
nsr_x = np.load('xgen0.npy')
pr_x = np.load('xgen1.npy')

In [33]:
len(nsr_x)

400

In [16]:
t1 = time.time()
for i in range(len(nsr_x)):
    cf = []
    cf.append(AE((nsr_x[i])))
    cf.append(SM2((nsr_x[i])))
    cf.append((LOG((nsr_x[i]))))
    cf.append((WL((nsr_x[i]))))
    cf.append((AC((nsr_x[i]))))
    cf.append(((BE((nsr_x[i])))))
    cf.append((SE((nsr_x[i]))))
    cf.append((TRAS((nsr_x[i]))))
    cf.append((VAR((nsr_x[i]))))
    cf.append((AAC((nsr_x[i]))))
    fx.append(cf)
    y_b.append(0)
    print('>', end = '')
t2 = time.time()
print(t2-t1)

c:\users\user\anaconda3\envs\ml\lib\site-packages\scipy\signal\spectral.py:1969: UserWarning: nperseg = 256 is greater than input length  = 1, using nperseg = 1
  .format(nperseg, input_length))
c:\users\user\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:75: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
c:\users\user\anaconda3\envs\ml\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\user\anaconda3\envs\ml\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>445.8790018558502


In [24]:
features = []
for f in fx:
    features.append([f[0][0], f[1][0], f[2], f[3], f[4], f[5], f[6], f[7], f[8], 0])

In [25]:
features

[[0.17398024,
  0.0,
  0.007158600946454315,
  0.0,
  0.19371425531738837,
  3.091653532621887,
  2.1988124015803923,
  1.3500674e-07,
  0.00018465311,
  0],
 [0.17184128,
  0.0,
  0.007181111175081835,
  0.0,
  0.19398281335356088,
  3.0923848391559683,
  2.199782483452001,
  1.3969766e-07,
  0.0001835553,
  0],
 [0.16852541,
  0.0,
  0.00712419327594099,
  0.0,
  0.1938917303293395,
  3.0801309835510104,
  2.1992973248838465,
  1.4499273e-07,
  0.00018170146,
  0],
 [0.16566204,
  0.0,
  0.0071302151601894724,
  0.0,
  0.19370788907306047,
  3.0701492017062195,
  2.197821244819405,
  1.49941e-07,
  0.00018002631,
  0],
 [0.16323143,
  0.0,
  0.007130242089359218,
  0.0,
  0.19343212732539744,
  3.0577121474077797,
  2.19667713028316,
  1.5455964e-07,
  0.00017852122,
  0],
 [0.161217,
  0.0,
  0.0070806386925204,
  0.0,
  0.19306493687466522,
  3.0471006677061427,
  2.19667713028316,
  1.5886314e-07,
  0.00017717895,
  0],
 [0.15960579,
  0.0,
  0.006960453262468451,
  0.0,
  0.19260

In [28]:
for f in fx:
    features.append([f[0][0], f[1][0], f[2], f[3], f[4], f[5], f[6], f[7], f[8], 0])

In [ ]:
fx = []
y_b = []

In [26]:
t1 = time.time()
for i in range(len(pr_x)):
    cf = []
    cf.append(AE((pr_x[i])))
    cf.append(SM2((pr_x[i])))
    cf.append((LOG((pr_x[i]))))
    cf.append((WL((pr_x[i]))))
    cf.append((AC((pr_x[i]))))
    cf.append(((BE((pr_x[i])))))
    cf.append((SE((pr_x[i]))))
    cf.append((TRAS((pr_x[i]))))
    cf.append((VAR((pr_x[i]))))
    cf.append((AAC((pr_x[i]))))
    fx.append(cf)
    y_b.append(1)
    print('>', end = '')
    
t2 = time.time()
print(t2-t1)

c:\users\user\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:75: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>424.5796272754669


In [27]:
fx

[[array([0.17398024], dtype=float32),
  array([0.], dtype=float32),
  0.007158600946454315,
  0.0,
  0.19371425531738837,
  3.091653532621887,
  2.1988124015803923,
  1.3500674e-07,
  0.00018465311,
  nan],
 [array([0.17184128], dtype=float32),
  array([0.], dtype=float32),
  0.007181111175081835,
  0.0,
  0.19398281335356088,
  3.0923848391559683,
  2.199782483452001,
  1.3969766e-07,
  0.0001835553,
  nan],
 [array([0.16852541], dtype=float32),
  array([0.], dtype=float32),
  0.00712419327594099,
  0.0,
  0.1938917303293395,
  3.0801309835510104,
  2.1992973248838465,
  1.4499273e-07,
  0.00018170146,
  nan],
 [array([0.16566204], dtype=float32),
  array([0.], dtype=float32),
  0.0071302151601894724,
  0.0,
  0.19370788907306047,
  3.0701492017062195,
  2.197821244819405,
  1.49941e-07,
  0.00018002631,
  nan],
 [array([0.16323143], dtype=float32),
  array([0.], dtype=float32),
  0.007130242089359218,
  0.0,
  0.19343212732539744,
  3.0577121474077797,
  2.19667713028316,
  1.5455964

In [30]:
len(features)

1200

In [31]:
len(fx)

800

In [32]:
len(y_b)

800

In [35]:
len(features[0])

10

In [37]:
f1 = features[0:400]
f2 = features[400:800]
f3 = features[800:1200]

In [38]:
f1 == f2

True

In [39]:
f1 == f3

False

In [40]:
f2 == f3

False

In [41]:
f = []
y = []
f.extend(f2)
f.extend(f3)
y.extend(np.zeros(400))
y.extend(np.ones(400))

In [43]:
len(f)

800

In [44]:
len(y)

800

In [45]:
y

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [46]:
f = np.array(f)
y = np.array(y)

In [47]:
np.save('Xf_aug.npy', f)
np.save('y_aug.npy', y)

In [48]:
nsr_x = np.load('xgen0.npy')
pr_x = np.load('xgen1.npy')

Xs_aug = np.concatenate((nsr_x, pr_x), axis = 0)

In [49]:
Xs_aug.shape

(800, 905, 1)

In [50]:
np.save('Xs_aug.npy', Xs_aug)